# Data Preparation and Feature Engineering

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
import pandas as pd
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.kernel_approximation import RBFSampler
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, balanced_accuracy_score, roc_auc_score, confusion_matrix
from sklearn.utils.fixes import loguniform
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from bank_marketing import *

## Import Dataset

In [3]:
bank_mkt = import_dataset("../data/BankMarketing.csv")

## Partition

In [37]:
def split_dataset(data, preprocessor=None, random_state=82):
    """
    Split dataset into train, test and validation sets using preprocessor.
    Because the random state of validation set is not specified, the validation set will be different each time when the function is called.

    Parameters
    ----------
        data : DataFrame

        preprocessor : Pipeline

    Returns
    -------
        datasets : tuple

    Examples
    --------
        from sklearn.preprocessing import OrdinalEncoder
        data = import_dataset("../data/BankMarketing.csv").interpolate(method="pad").loc[:, ["job", "education", "y"]]
        # To unpack all train, test, and validation sets 
        X_train, y_train, X_test, y_test, X_ttrain, y_ttrain, X_validate, y_validate = split_dataset(data, OrdinalEncoder())
        # To unpack train and test sets.
        X_train, y_train, X_test, y_test, *other_sets = split_dataset(data, OrdinalEncoder())
        # To unpack test and validation set
        *other_sets, X_test, y_test, X_ttrain, y_ttrain, X_validate, y_validate = split_dataset(data, OrdinalEncoder())
        # To unpack only train set.
        X_train, y_train, *other_sets = split_dataset(data, OneHotEncoder())
    """
    train_test_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
    for train_index, test_index in train_test_split.split(data.drop("y", axis=1), data["y"]):
        train_set = data.iloc[train_index]
        test_set = data.iloc[test_index]

    X_train = train_set.drop(["duration", "y"], axis=1)
    y_train = train_set["y"].astype("int").to_numpy()
    X_test = test_set.drop(["duration", "y"], axis=1)
    y_test = test_set["y"].astype("int").to_numpy()

    train_validate_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
    for ttrain_index, validate_index in train_validate_split.split(X_train, y_train):
        ttrain_set = train_set.iloc[ttrain_index]
        validate_set = train_set.iloc[validate_index]
    
    X_ttrain = ttrain_set.drop(["duration", "y"], axis=1)
    y_ttrain = ttrain_set["y"].astype("int").to_numpy()
    X_validate = validate_set.drop(["duration", "y"], axis=1)
    y_validate = validate_set["y"].astype("int").to_numpy()

    if preprocessor != None:
        X_train = preprocessor.fit_transform(X_train, y_train)
        X_test = preprocessor.transform(X_test)
        X_ttrain = preprocessor.fit_transform(X_ttrain, y_ttrain)
        X_validate = preprocessor.transform(X_validate)

    return (X_train, y_train, X_test, y_test, X_ttrain, y_ttrain, X_validate, y_validate)

In [38]:
def benchmark(data, preprocessor=None, clf=None):
    """
    Benchmark preprocessor and clf's performance on train, validation and test sets. 
    All the data transformation should be handled by preprocessor and estimation should be handled by clf.
    
    Parameters
    ----------
        data : DataFrame
        
        preprocessor : Pipeline, default = None
        
        clf : estimator, default = None
        
    """
    X_train, y_train, X_test, y_test, X_ttrain, y_ttrain, X_validate, y_validate = split_dataset(data, preprocessor)
    
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.to_numpy()
        X_test = X_test.to_numpy()
        X_ttrain = X_ttrain.to_numpy()
        X_validate = X_validate.to_numpy()
    
    X_sets = [X_ttrain, X_validate, X_test]
    y_sets = [y_ttrain, y_validate, y_test]
    
    metric_names = ["TNR", "TPR", "bACC", "ROC", "REC", "PRE", "AP"]
    set_names = ["Train", "Validate", "Test"]
    metric_df = pd.DataFrame(index=metric_names, columns=set_names)
            
    try:
        clf.fit(X_ttrain, y_ttrain, eval_set=(X_validate, y_validate), verbose=False)
    except (ValueError, TypeError):
        clf.fit(X_ttrain, y_ttrain)
        
    for name, X, y in zip(set_names, X_sets, y_sets):
        # Re-fit model on train set before test set evaluation except CatBoost
        if name == "Test" and not isinstance(clf, CatBoostClassifier):
            clf.fit(X_train, y_train)
        y_pred = clf.predict(X)

        try:
            y_score = clf.decision_function(X)
        except AttributeError:
            y_score = clf.predict_proba(X)[:, 1]
            
        metrics = [recall_score(y, y_pred, pos_label=0),
                   recall_score(y, y_pred),
                   balanced_accuracy_score(y, y_pred),
                   roc_auc_score(y, y_score),
                   recall_score(y, y_pred),
                   precision_score(y, y_pred),
                   average_precision_score(y, y_score)]
        metric_df[name] = metrics
        
    return metric_df

In [47]:
def dftransform(X,
                drop=None,
                cut=None,
                gen=None,
                cyclic=None,
                target=None,
                fillna=True,
                to_numpy=False):
    """
    Encode, transform, and generate categorical data in the dataframe.
    
    Parameters
    ----------
        X : DataFrame
        
        drop : list, default = None
        
        gen : list, default = None
        
        cut : list, default = None
        
        external : list, default = None
        
        cyclic : list, default = None

        fillna : boolean, default = True
    
    Returns
    -------
        X : DataFrame

    Examples
    --------
    bank_mkt = import_dataset("../data/BankMarketing.csv")
    X = cat_encode(bank_mkt)
    """
    X = X.copy()
    
    if gen != None:
        if "year" in gen or "days" in gen:
            X.loc[X.index < 27682, "year"] = 2008
            X.loc[(27682<=X.index) & (X.index<39118), "year"] = 2009
            X.loc[39118<=X.index, "year"] = 2010
            X["year"] = X["year"].astype("int")
        if "days" in gen:
            X["date"] = pd.to_datetime(X[["month", "year"]].assign(day=1))
            X["lehman"] = pd.to_datetime("2008-09-15")
            X["days"] = X["date"] -  X["lehman"]
            X["days"] = X["days"].dt.days
#             X = X.drop(["lehman", "year", "date"], axis=1)
        if "has_previous" in gen:
            X["has_previous"] = X["previous"] > 0
        if "has_default" in gen:
            X["has_default"] = X["default"].notna()
        if "has_marital" in gen:
            X["has_marital"] = X["marital"].notna()

    if cut != None:
        if "pdays" in cut:
            # Cut pdays into categories
            X["pdays"] = pd.cut(X["pdays"], [0, 3, 5, 10, 15, 30, 1000], labels=[3, 5, 10, 15, 30, 1000], include_lowest=True).astype("Int64")
    
    if cyclic != None:
        if "month" in cyclic:
            X['month_sin'] = np.sin(2 * np.pi * X["month"]/12)
            X['month_cos'] = np.cos(2 * np.pi * X["month"]/12)
            X = X.drop("month", axis=1)
        if "day_of_week" in cyclic:
            X['day_sin'] = np.sin(2 * np.pi * X["day_of_week"]/5)
            X['day_cos'] = np.cos(2 * np.pi * X["day_of_week"]/5)
            X = X.drop("day_of_week", axis=1)
            
    # Transform target encoded feature as str
    if target != None:
        X[target] = X[target].astype("str")

    # Other categorical features will be coded as its order in pandas categorical index
    X = X.apply(lambda x: x.cat.codes if pd.api.types.is_categorical_dtype(x) else (x.astype("Int64") if pd.api.types.is_bool_dtype(x) else x))
    
    if fillna:
        # Clients who have been contacted but do not have pdays record should be encoded as 999
        # Clients who have not been contacted should be encoded as -999 
        X.loc[X["pdays"].isna() & X["poutcome"].notna(), "pdays"] = 999
        X["pdays"] = X["pdays"].fillna(-999)
        # Fill other missing values as -1
        X = X.fillna(-1)
    else:
        X = X.astype("float")
    
    if drop != None:
        # Drop features
        X = X.drop(drop, axis=1)
    
    if to_numpy:
        X = X.to_numpy()
    
    return X

In [49]:
dftransform(bank_mkt).to_csv("bank.csv")

## Baseline

In [40]:
dftransformer = FunctionTransformer(dftransform, kw_args={"to_numpy":True})
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1, 8])
cat_baseline = benchmark(bank_mkt, dftransformer, cat_clf)
cat_baseline

,Train,Validate,Test
TNR,0.862592,0.857338,0.860290
TPR,0.662513,0.657682,0.646552
bACC,0.762553,0.757510,0.753421
ROC,0.849549,0.808328,0.801834
REC,0.662513,0.657682,0.646552
PRE,0.379730,0.369138,0.370142
AP,0.536392,0.483245,0.446779


In [21]:
cat_features = ["job",
                "marital",
                "education",
                "default",
                "housing",
                "loan",
                "poutcome"]

num_features =  ["age",
                 "campaign",
                 "pdays",
                 "previous",
                 "emp.var.rate",
                 "cons.price.idx",
                 "cons.conf.idx",
                 "euribor3m",
                 "nr.employed"]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

cat_transformer = make_pipeline(FunctionTransformer(dftransform), hot_scaler)

In [22]:
logit_clf = LogisticRegression(penalty="none", class_weight="balanced", max_iter=1000)
logit_baseline = benchmark(bank_mkt, cat_transformer, logit_clf)
logit_baseline

,Train,Validate,Test
TNR,0.803019,0.797639,0.808292
TPR,0.660155,0.650943,0.682112
bACC,0.731587,0.724291,0.745202
ROC,0.782788,0.788192,0.791102
REC,0.660155,0.650943,0.682112
PRE,0.298507,0.289916,0.311209
AP,0.439192,0.447445,0.440707


In [23]:
rbf_clf = Pipeline([
    ("rbf", RBFSampler(gamma=0.001, random_state=42)),
    ("svm", LinearSVC(C=1, loss="squared_hinge", dual=False, class_weight="balanced", max_iter=1000))
])
rbf_baseline = benchmark(bank_mkt, cat_transformer, rbf_clf)
rbf_baseline

,Train,Validate,Test
TNR,0.798871,0.806534,0.798030
TPR,0.666891,0.644205,0.693966
bACC,0.732881,0.725370,0.745998
ROC,0.790151,0.771573,0.793617
REC,0.666891,0.644205,0.693966
PRE,0.296274,0.297079,0.303774
AP,0.448081,0.413439,0.441643


## Drop Client Data

In [31]:
# Plot Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# cosmetic options
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("png")
rc = {"figure.figsize": (6.4, 4.8),
      "figure.dpi": 300,
      "axes.titlesize": "large",
      "axes.titleweight": "bold",
      "axes.titlepad": 12,
      "axes.titlelocation": "left"}
sns.set_theme(context="notebook", style="darkgrid", color_codes=True, rc=rc)

In [41]:
drop_features = ["age",
                 "job",
                 "marital",
                 "education",
                 "housing",
                 "loan",
                 "default"]

drop_encoder = FunctionTransformer(dftransform, kw_args={"drop": drop_features,
                                                         "to_numpy":True})

In [42]:
cat_drop = benchmark(bank_mkt, drop_encoder, cat_clf)
drop_result = pd.concat([cat_baseline, cat_drop], keys=["CatBoost Baseline", "CatBoost Drop"], axis=1)
drop_result.to_csv("drop.csv")

In [43]:
drop_result

CatBoost Baseline                     CatBoost Drop                    
                 Train  Validate      Test         Train  Validate      Test
TNR           0.862592  0.857338  0.860290      0.862935  0.851009  0.858785
TPR           0.662513  0.657682  0.646552      0.654429  0.615903  0.644397
bACC          0.762553  0.757510  0.753421      0.758682  0.733456  0.751591
ROC           0.849549  0.808328  0.801834      0.825988  0.800781  0.803795
REC           0.662513  0.657682  0.646552      0.654429  0.615903  0.644397
PRE           0.379730  0.369138  0.370142      0.377428  0.344127  0.366871
AP            0.536392  0.483245  0.446779      0.514834  0.463497  0.452306

In [102]:
cat_features = ["poutcome"]

num_features =  ["campaign",
                 "pdays",
                 "previous",
                 "emp.var.rate",
                 "cons.price.idx",
                 "cons.conf.idx",
                 "euribor3m",
                 "nr.employed"]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

drop_transformer = make_pipeline(drop_encoder, hot_scaler)

logit_drop = benchmark(bank_mkt, drop_transformer, logit_clf)
pd.concat([logit_baseline, logit_drop], keys=["Logistic Baseline", "Logistic Drop"], axis=1)

In [104]:
rbf_drop = benchmark(bank_mkt, drop_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_drop], keys=["RBF Baseline", "RBF Drop"], axis=1)

RBF Baseline                      RBF Drop                    
            Train  Validate      Test     Train  Validate      Test
TNR      0.731355  0.727626  0.482710  0.699376  0.696984  0.446551
TPR      0.519048  0.495238  0.433635  0.573214  0.495238  0.459236
bACC     0.625202  0.611432  0.458173  0.636295  0.596111  0.452893
ROC      0.682524  0.657794  0.457365  0.680859  0.650337  0.480152
REC      0.519048  0.495238  0.433635  0.573214  0.495238  0.459236
PRE      0.116267  0.110169  0.271986  0.114916  0.100144  0.269970
AP       0.190419  0.185811  0.297581  0.182002  0.162034  0.349300

## Drop Everything Except Client Data

In [105]:
drop_features = ["contact",
                 "month",
                 "day_of_week",
                 "duration",
                 "campaign",
                 "pdays",
                 "previous",
                 "poutcome",
                 "emp.var.rate",
                 "cons.price.idx",
                 "cons.conf.idx",
                 "euribor3m",
                 "duration",
                 "y"]

client_encoder = FunctionTransformer(cat_encode, kw_args={"drop": drop_features})

In [106]:
cat_client = benchmark(bank_mkt, client_encoder, cat_clf)
pd.concat([cat_baseline, cat_client], keys=["Catboost Baseline", "Catboost Client"], axis=1)

Catboost Baseline                     Catboost Client                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.946782  0.943256  0.903633        0.924206  0.923800  0.262243
TPR           0.327976  0.302381  0.333596        0.284524  0.326190  0.865301
bACC          0.637379  0.622818  0.618615        0.604365  0.624995  0.563772
ROC           0.749520  0.680977  0.680787        0.696060  0.680970  0.594448
REC           0.327976  0.302381  0.333596        0.284524  0.326190  0.865301
PRE           0.295601  0.266247  0.606734        0.203578  0.225700  0.343281
AP            0.267692  0.216556  0.499409        0.177429  0.184081  0.378059

In [107]:
cat_features = ["job",
                "marital",
                "education",
                "default",
                "housing",
                "loan"]

num_features =  ["age"]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

client_transformer = make_pipeline(client_encoder, hot_scaler)


In [108]:
logit_client = benchmark(bank_mkt, client_transformer, logit_clf)
pd.concat([logit_baseline, logit_client], keys=["Logistic Baseline", "Logistic Client"], axis=1)

Logistic Baseline                     Logistic Client                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.749838  0.750649  0.555731        0.495379  0.481031  0.339652
TPR           0.507143  0.466667  0.315479        0.637500  0.628571  0.780228
bACC          0.628490  0.608658  0.435605        0.566440  0.554801  0.559940
ROC           0.684280  0.671550  0.388754        0.591552  0.582950  0.574904
REC           0.507143  0.466667  0.315479        0.637500  0.628571  0.780228
PRE           0.121298  0.113033  0.240396        0.079210  0.076190  0.344942
AP            0.195585  0.184663  0.305101        0.089746  0.088462  0.345087

In [109]:
rbf_client = benchmark(bank_mkt, client_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_client], keys=["RBF Baseline", "RBF Client"], axis=1)

RBF Baseline                     RBF Client                    
            Train  Validate      Test      Train  Validate      Test
TNR      0.731355  0.727626  0.482710   0.783358  0.781615  0.320520
TPR      0.519048  0.495238  0.433635   0.454762  0.447619  0.386373
bACC     0.625202  0.611432  0.458173   0.619060  0.614617  0.353446
ROC      0.682524  0.657794  0.457365   0.666659  0.653124  0.323596
REC      0.519048  0.495238  0.433635   0.454762  0.447619  0.386373
PRE      0.116267  0.110169  0.271986   0.125061  0.122476  0.202185
AP       0.190419  0.185811  0.297581   0.144859  0.139881  0.225061

## Impute Missing Values

### Use Most Frequent Values

In [110]:
freq_features = ["job", "marital", "education", "default", "housing", "loan"]

freq_imputer = ColumnTransformer([
    ("freq_imputer", SimpleImputer(missing_values=-1, strategy="most_frequent"), freq_features)
], remainder="passthrough")

freq_encoder = make_pipeline(cat_encoder, freq_imputer)

In [111]:
cat_freq = benchmark(bank_mkt, freq_encoder, cat_clf)
pd.concat([cat_baseline, cat_freq], keys=["Catboost Baseline", "Catboost Frequent"], axis=1)

Catboost Baseline                     Catboost Frequent            \
                 Train  Validate      Test             Train  Validate   
TNR           0.946782  0.943256  0.903633          0.953753  0.952497   
TPR           0.327976  0.302381  0.333596          0.326190  0.290476   
bACC          0.637379  0.622818  0.618615          0.639972  0.621486   
ROC           0.749520  0.680977  0.680787          0.764044  0.697451   
REC           0.327976  0.302381  0.333596          0.326190  0.290476   
PRE           0.295601  0.266247  0.606734          0.324452  0.293976   
AP            0.267692  0.216556  0.499409          0.287825  0.204977   

                
          Test  
TNR   0.873793  
TPR   0.348169  
bACC  0.610981  
ROC   0.693952  
REC   0.348169  
PRE   0.551466  
AP    0.497987

In [112]:
# Select "job", "marital", "education"
cat_features = [0,1,2]

# Select "age", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"
num_features = [5,10,11,12,14,15,16,17,18]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

freq_transformer = make_pipeline(cat_encoder, freq_imputer, hot_scaler)


In [113]:
logit_freq = benchmark(bank_mkt, freq_transformer, logit_clf)
pd.concat([logit_baseline, logit_freq], keys=["Logistic Baseline", "Logistic Frequent"], axis=1)

/usr/local/Caskroom/miniconda/base/envs/analytics/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Baseline                     Logistic Frequent            \
                 Train  Validate      Test             Train  Validate   
TNR           0.749838  0.750649  0.555731          0.769374  0.776265   
TPR           0.507143  0.466667  0.315479          0.484524  0.478571   
bACC          0.628490  0.608658  0.435605          0.626949  0.627418   
ROC           0.684280  0.671550  0.388754          0.681100  0.685639   
REC           0.507143  0.466667  0.315479          0.484524  0.478571   
PRE           0.121298  0.113033  0.240396          0.125154  0.127135   
AP            0.195585  0.184663  0.305101          0.190328  0.190995   

                
          Test  
TNR   0.541689  
TPR   0.317448  
bACC  0.429568  
ROC   0.388679  
REC   0.317448  
PRE   0.235879  
AP    0.304329

In [114]:
rbf_freq = benchmark(bank_mkt, freq_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_freq], keys=["RBF Baseline", "RBF Frequent"], axis=1)

RBF Baseline                     RBF Frequent                    
            Train  Validate      Test        Train  Validate      Test
TNR      0.731355  0.727626  0.482710     0.740880  0.736057  0.391610
TPR      0.519048  0.495238  0.433635     0.499405  0.538095  0.581331
bACC     0.625202  0.611432  0.458173     0.620143  0.637076  0.486470
ROC      0.682524  0.657794  0.457365     0.672820  0.696074  0.527953
REC      0.519048  0.495238  0.433635     0.499405  0.538095  0.581331
PRE      0.116267  0.110169  0.271986     0.116012  0.121899  0.298665
AP       0.190419  0.185811  0.297581     0.185224  0.199780  0.399867

### Estimate Missing Values 

In [115]:
ite_features = ["age", "job", "marital", "education", "default", "housing", "loan", "contact", "campaign", "month", "day_of_week", "pdays", "previous"]

ite_imputer = ColumnTransformer([
    ("ite_imputer",
     make_pipeline(
         IterativeImputer(max_iter=100, missing_values=-1, initial_strategy="most_frequent", random_state=42),
         FunctionTransformer(np.round)
     ),
     ite_features),
], remainder="passthrough")

ite_encoder = make_pipeline(cat_encoder, ite_imputer)

In [116]:
cat_ite = benchmark(bank_mkt, ite_encoder, cat_clf)
pd.concat([cat_baseline, cat_ite], keys=["Catboost Baseline", "Catboost Iterative"], axis=1)

Catboost Baseline                     Catboost Iterative            \
                 Train  Validate      Test              Train  Validate   
TNR           0.946782  0.943256  0.903633           0.950186  0.945687   
TPR           0.327976  0.302381  0.333596           0.295238  0.359524   
bACC          0.637379  0.622818  0.618615           0.622712  0.652606   
ROC           0.749520  0.680977  0.680787           0.721813  0.721429   
REC           0.327976  0.302381  0.333596           0.295238  0.359524   
PRE           0.295601  0.266247  0.606734           0.287536  0.310700   
AP            0.267692  0.216556  0.499409           0.245008  0.263722   

                
          Test  
TNR   0.725996  
TPR   0.554943  
bACC  0.640470  
ROC   0.707384  
REC   0.554943  
PRE   0.474411  
AP    0.490075

In [117]:
# Select "job", "marital", "education", "default", "housing", "loan", "poutcome"
cat_features = [1,2,3,4,5,13]

# Select "age", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"
num_features = [0,8,9,10,12,14,15,16,17,18]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

ite_transformer = make_pipeline(cat_encoder, ite_imputer, hot_scaler)

In [118]:
logit_ite = benchmark(bank_mkt, ite_transformer, logit_clf)
pd.concat([logit_baseline, logit_ite], keys=["Logistic Baseline", "Logistic Iterative"], axis=1)

Logistic Baseline                     Logistic Iterative            \
                 Train  Validate      Test              Train  Validate   
TNR           0.749838  0.750649  0.555731           0.712751  0.700713   
TPR           0.507143  0.466667  0.315479           0.542857  0.545238   
bACC          0.628490  0.608658  0.435605           0.627804  0.622976   
ROC           0.684280  0.671550  0.388754           0.681547  0.670371   
REC           0.507143  0.466667  0.315479           0.542857  0.545238   
PRE           0.121298  0.113033  0.240396           0.114014  0.110361   
AP            0.195585  0.184663  0.305101           0.187467  0.189898   

                
          Test  
TNR   0.525364  
TPR   0.315085  
bACC  0.420224  
ROC   0.387802  
REC   0.315085  
PRE   0.228311  
AP    0.301043

In [119]:
rbf_ite = benchmark(bank_mkt, ite_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_ite], keys=["RBF Baseline", "RBF Iterative"], axis=1)

RBF Baseline                     RBF Iterative                    
            Train  Validate      Test         Train  Validate      Test
TNR      0.731355  0.727626  0.482710      0.712468  0.712873  0.464279
TPR      0.519048  0.495238  0.433635      0.519048  0.564286  0.517133
bACC     0.625202  0.611432  0.458173      0.615758  0.638579  0.490706
ROC      0.682524  0.657794  0.457365      0.668423  0.690162  0.528076
REC      0.519048  0.495238  0.433635      0.519048  0.564286  0.517133
PRE      0.116267  0.110169  0.271986      0.109465  0.118028  0.300802
AP       0.190419  0.185811  0.297581      0.177775  0.233388  0.396487

## Feature Engineering `pdays`

In [32]:
cut_encoder = FunctionTransformer(cat_encode, kw_args={"cut": ["pdays"]})

In [33]:
cat_cut = benchmark(bank_mkt, cut_encoder, cat_clf)
pd.concat([cat_baseline, cat_cut], keys=["Catboost Baseline", "Catboost Cut"], axis=1)

Catboost Baseline                     Catboost Cut                    
                 Train  Validate      Test        Train  Validate      Test
TNR           0.888612  0.891576  0.020305     0.883267  0.887498  0.113706
TPR           0.554154  0.520337  0.992544     0.535226  0.535764  0.966449
bACC          0.721383  0.705957  0.506424     0.709247  0.711631  0.540077
ROC           0.822153  0.767514  0.704615     0.777547  0.771101  0.709818
REC           0.554154  0.520337  0.992544     0.535226  0.535764  0.966449
PRE           0.332912  0.324869  0.524631     0.315040  0.323181  0.542932
AP            0.417707  0.361904  0.730569     0.368836  0.347587  0.717697

In [34]:
cat_features = ["job",
                "marital",
                "education",
                "default",
                "housing",
                "loan",
                "poutcome",
                "pdays"]

num_features = ["age", "campaign", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

cut_transformer = make_pipeline(cut_encoder, hot_scaler)

In [35]:
logit_cut = benchmark(bank_mkt, cut_transformer, logit_clf)
pd.concat([logit_baseline, logit_cut], keys=["Logistic Baseline", "Logistic Cut"], axis=1)

Logistic Baseline                     Logistic Cut                    
                 Train  Validate      Test        Train  Validate      Test
TNR           0.761401  0.764309  0.463959     0.763405  0.770356  0.498477
TPR           0.621101  0.615708  0.811743     0.617596  0.614306  0.795899
bACC          0.691251  0.690009  0.637851     0.690500  0.692331  0.647188
ROC           0.746054  0.733595  0.718543     0.745403  0.741580  0.718480
REC           0.621101  0.615708  0.811743     0.617596  0.614306  0.795899
PRE           0.207058  0.207565  0.622588     0.207514  0.211492  0.633531
AP            0.321821  0.312369  0.716084     0.321846  0.317145  0.715623

In [36]:
rbf_ite = benchmark(bank_mkt, ite_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_ite], keys=["RBF Baseline", "RBF Iterative"], axis=1)

RBF Baseline                     RBF Iterative                    
            Train  Validate      Test         Train  Validate      Test
TNR      0.779157  0.777247  0.160406      0.770824  0.756856  0.165482
TPR      0.608833  0.586255  0.954334      0.609884  0.615708  0.962721
bACC     0.693995  0.681751  0.557370      0.690354  0.686282  0.564102
ROC      0.750144  0.735048  0.728434      0.746573  0.741733  0.720813
REC      0.608833  0.586255  0.954334      0.609884  0.615708  0.962721
PRE      0.216638  0.208791  0.553214      0.210705  0.202491  0.556873
AP       0.321448  0.299921  0.731069      0.312602  0.319220  0.697359

## Cyclic Encoding

In [37]:
cyclic_encoder = FunctionTransformer(cat_encode, kw_args={"cyclic":["month", "day_of_week"]})

In [38]:
cat_cyclic = benchmark(bank_mkt, cyclic_encoder, cat_clf)
pd.concat([cat_baseline, cat_cyclic], keys=["Catboost Baseline", "Catboost Cyclic"], axis=1)

Catboost Baseline                     Catboost Cyclic                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.888612  0.891576  0.020305        0.884216  0.887639  0.017259
TPR           0.554154  0.520337  0.992544        0.547844  0.521739  0.996272
bACC          0.721383  0.705957  0.506424        0.716030  0.704689  0.506766
ROC           0.822153  0.767514  0.704615        0.791883  0.748530  0.732023
REC           0.554154  0.520337  0.992544        0.547844  0.521739  0.996272
PRE           0.332912  0.324869  0.524631        0.321870  0.317677  0.524791
AP            0.417707  0.361904  0.730569        0.387561  0.338399  0.724835

In [39]:
cat_features = ["job",
                "marital",
                "education",
                "default",
                "housing",
                "loan",
                "poutcome"]

num_features = ["age", "campaign", "pdays", "previous",
                "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed",
                "month_sin", "month_cos", "day_sin", "day_cos"]

hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

cyclic_transformer = make_pipeline(cyclic_encoder, hot_scaler)

In [40]:
logit_ite = benchmark(bank_mkt, cyclic_transformer, logit_clf)
pd.concat([logit_baseline, logit_ite], keys=["Logistic Baseline", "Logistic Cyclic"], axis=1)

Logistic Baseline                     Logistic Cyclic                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.761401  0.764309  0.463959        0.775078  0.777809  0.007107
TPR           0.621101  0.615708  0.811743        0.616544  0.612903  0.996272
bACC          0.691251  0.690009  0.637851        0.695811  0.695356  0.501689
ROC           0.746054  0.733595  0.718543        0.749987  0.739776  0.632209
REC           0.621101  0.615708  0.811743        0.616544  0.612903  0.996272
PRE           0.207058  0.207565  0.622588        0.215669  0.216658  0.522228
AP            0.321821  0.312369  0.716084        0.322118  0.303164  0.652721

In [41]:
rbf_ite = benchmark(bank_mkt, cyclic_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_ite], keys=["RBF Baseline", "RBF Cyclic"], axis=1)

RBF Baseline                     RBF Cyclic                    
            Train  Validate      Test      Train  Validate      Test
TNR      0.779157  0.777247  0.160406   0.777293  0.769793  0.000000
TPR      0.608833  0.586255  0.954334   0.607431  0.638149  0.998136
bACC     0.693995  0.681751  0.557370   0.692362  0.703971  0.499068
ROC      0.750144  0.735048  0.728434   0.748003  0.747419  0.652762
REC      0.608833  0.586255  0.954334   0.607431  0.638149  0.998136
PRE      0.216638  0.208791  0.553214   0.214826  0.217495  0.520914
AP       0.321448  0.299921  0.731069   0.311819  0.344888  0.660708

## Mean Encoding

In [42]:
target_features = ["job", "marital", "education", "month", "day_of_week"] 

target_encoder = make_pipeline(FunctionTransformer(cat_encode, kw_args={"target": target_features}), TargetEncoder(cols=target_features))

In [43]:
cat_target = benchmark(bank_mkt, target_encoder, cat_clf)
pd.concat([cat_baseline, cat_target], keys=["Catboost Baseline", "Catboost Target"], axis=1)

Catboost Baseline                     Catboost Target                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.888612  0.891576  0.020305        0.889385  0.881733  0.207107
TPR           0.554154  0.520337  0.992544        0.558009  0.568022  0.941286
bACC          0.721383  0.705957  0.506424        0.723697  0.724877  0.574196
ROC           0.822153  0.767514  0.704615        0.823417  0.772176  0.704155
REC           0.554154  0.520337  0.992544        0.558009  0.568022  0.941286
PRE           0.332912  0.324869  0.524631        0.336007  0.325040  0.563931
AP            0.417707  0.361904  0.730569        0.424982  0.350502  0.712325

In [44]:
num_features = [0,1,2,3,8,9,11,14,15,16,17,18]

target_scaler = ColumnTransformer([
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

target_transformer = make_pipeline(target_encoder, target_scaler)

In [45]:
logit_target = benchmark(bank_mkt, target_transformer, logit_clf)
pd.concat([logit_baseline, logit_target], keys=["Logistic Baseline", "Logistic Target"], axis=1)

Logistic Baseline                     Logistic Target                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.761401  0.764309  0.463959        0.785451  0.779918  0.205076
TPR           0.621101  0.615708  0.811743        0.620750  0.589060  0.940354
bACC          0.691251  0.690009  0.637851        0.703100  0.684489  0.572715
ROC           0.746054  0.733595  0.718543        0.752667  0.728992  0.701907
REC           0.621101  0.615708  0.811743        0.620750  0.589060  0.940354
PRE           0.207058  0.207565  0.622588        0.224946  0.211587  0.563058
AP            0.321821  0.312369  0.716084        0.326746  0.314612  0.690270

In [46]:
rbf_target = benchmark(bank_mkt, target_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_target], keys=["RBF Baseline", "RBF Target"], axis=1)

RBF Baseline                     RBF Target                    
            Train  Validate      Test      Train  Validate      Test
TNR      0.779157  0.777247  0.160406   0.795436  0.790184  0.118782
TPR      0.608833  0.586255  0.954334   0.609884  0.587658  0.972973
bACC     0.693995  0.681751  0.557370   0.702660  0.688921  0.545877
ROC      0.750144  0.735048  0.728434   0.755447  0.733983  0.642283
REC      0.608833  0.586255  0.954334   0.609884  0.587658  0.972973
PRE      0.216638  0.208791  0.553214   0.230220  0.219257  0.546025
AP       0.321448  0.299921  0.731069   0.328199  0.294682  0.622557

## Entity Embeddings

## External Infomation

CPI: http://www.ine.pt

In [163]:
drop_features = ['age',
                 'job',
                 'marital',
                 'education',
                 'default',
                 'housing',
                 'loan',
                 "duration",
                 "y"]

year_encoder = FunctionTransformer(cat_encode, kw_args= {"drop": drop_features,
                                                         "external":["year", "date"]})

In [200]:
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,8])
cat_year = benchmark(bank_mkt, target_encoder, cat_clf)
pd.concat([cat_baseline, cat_year], keys=["Catboost Baseline", "Catboost Date"], axis=1)

Catboost Baseline                     Catboost Date                    
                 Train  Validate      Test         Train  Validate      Test
TNR           0.865201  0.862641  0.861932      0.867126  0.872562  0.862069
TPR           0.657797  0.630728  0.639009      0.646009  0.630728  0.634698
bACC          0.761499  0.746684  0.750470      0.756567  0.751645  0.748384
ROC           0.845568  0.797931  0.801313      0.824377  0.801862  0.799998
REC           0.657797  0.630728  0.639009      0.646009  0.630728  0.634698
PRE           0.382566  0.368214  0.370162      0.381692  0.385820  0.368817
AP            0.532276  0.467858  0.445780      0.510141  0.486803  0.448976

In [96]:
cat_features = ["job",
                "marital",
                "education",
                "default",
                "housing",
                "loan",
                "poutcome"]

num_features = ["age",
                "campaign", 
                "pdays",
                "previous",
                "emp.var.rate",
                "cons.price.idx",
                "cons.conf.idx",
                "euribor3m",
                "nr.employed"]
target_scaler = ColumnTransformer([
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")
                
hot_scaler = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(drop="first"), cat_features),
    ("scaler", StandardScaler(), num_features)
], remainder="passthrough")

target_transformer = make_pipeline(year_encoder, TargetEncoder(cols=target_features), hot_scaler)

In [97]:
logit_target = benchmark(bank_mkt, target_transformer, logit_clf)
pd.concat([logit_baseline, logit_target], keys=["Logistic Baseline", "Logistic Target"], axis=1)

Logistic Baseline                     Logistic Target                    
                 Train  Validate      Test           Train  Validate      Test
TNR           0.802036  0.795758  0.807608        0.802506  0.795587  0.807745
TPR           0.663860  0.663073  0.665948        0.666554  0.650943  0.665948
bACC          0.732948  0.729415  0.736778        0.734530  0.723265  0.736847
ROC           0.785500  0.784058  0.781970        0.788252  0.775852  0.781969
REC           0.663860  0.663073  0.665948        0.666554  0.650943  0.665948
PRE           0.298636  0.291815  0.305336        0.299985  0.287843  0.305487
AP            0.438799  0.458944  0.425859        0.445736  0.430392  0.425832

In [99]:
rbf_target = benchmark(bank_mkt, target_transformer, rbf_clf)
pd.concat([rbf_baseline, rbf_target], keys=["RBF Baseline", "RBF Target"], axis=1)

RBF Baseline                     RBF Target                    
            Train  Validate      Test      Train  Validate      Test
TNR      0.792627  0.795074  0.797345   0.790788  0.792508  0.799398
TPR      0.671270  0.671159  0.678879   0.672280  0.671159  0.676724
bACC     0.731948  0.733116  0.738112   0.731534  0.731833  0.738061
ROC      0.787844  0.792044  0.785194   0.791481  0.779615  0.781676
REC      0.671270  0.671159  0.678879   0.672280  0.671159  0.676724
PRE      0.291289  0.293632  0.298437   0.289779  0.291058  0.299904
AP       0.446463  0.451790  0.425436   0.452065  0.418888  0.425548

## Days Since Financial Crisis

In [116]:
drop_features = ["age",
                 "job",
                 "marital",
                 "education",
                 "housing",
                 "loan",
                 "default",
                 "duration",
                 "y"]

X_train, y_train, X_test, y_test, *other_sets = split_dataset(bank_mkt, 
                                                              FunctionTransformer(cat_encode, kw_args={"drop": drop_features,
                                                                                                       "gen": ["days"]}),
                                                              random_state=42,
                                                              to_numpy=False)
X_train = X_train.astype("float")
X_test = X_test.astype("float")
xgb_clf = XGBClassifier(max_depth=3, gamma=1, min_child_weight=1, scale_pos_weight=8)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
y_score = xgb_clf.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_score), average_precision_score(y_test, y_score))

0.8149101185757697 0.4863343642317203


## External Data

- Spending now or invest for the future
- price, inflation
- income, employment, labour supply
- comsuption level, GDP
- Composite leading indicator
- adjust interest rate
- Portugal bond return and rating
- German bond return
- Stock market return and volatity
- Gold price

## Upsampling by Year

In [ ]:
def upsampling_year(X):
    X = X.copy()
    X.loc[X.index < 27682, "year"] = 2008
    X.loc[(27682<=X.index) & (X.index<39118), "year"] = 2009
    X.loc[39118<=X.index, "year"] = 2010
    X["year"] = X["year"].astype("int")
    X_2008 = X[X["year"]==2008]
    n_2008 = X_2008.shape[0]
    X_2009 = resample(X[X["year"]==2009], replace=True, n_samples=n_2008, random_state=42)
    X_2010 = resample(X[X["year"]==2010], replace=True, n_samples=n_2008, random_state=42)
    X_upsampled = pd.concat([X_2008, X_2009, X_2010])
    return X_upsampled

In [ ]:
train_test_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in train_test_split.split(bank_mkt.drop("y", axis=1), bank_mkt["y"]):
    train_set = bank_mkt.iloc[train_index]
    test_set = bank_mkt.iloc[test_index]
drop_features=["duration", "y"]
preprocessor = FunctionTransformer(cat_encode, kw_args={"drop": drop_features, "gen": ["year", "days"]})
train_upsampled = upsampling_year(train_set)
y_train = train_upsampled["y"].astype("int").to_numpy()
y_test = test_set["y"].astype("int").to_numpy()
X_train = preprocessor.fit_transform(train_upsampled).astype("float")
X_test = preprocessor.transform(test_set).astype("float")
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,3], verbose=False)
xgb_clf = XGBClassifier(max_depth=5, gamma=1, min_child_weight=1, scale_pos_weight=1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
y_score = xgb_clf.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_score), average_precision_score(y_test, y_score))

## Summary